In [1]:
from envs.train_env import Electric_Car
from models.baseline import BaselineModel
from models.baseline2 import BaselineModel2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from models.tabular_qlearning import TabularQLearning
from envs.feature_engineering import DataHelper

In [2]:
dh = DataHelper()

def learn(env, model):
    states, rewards, infos = [], [], []
    truncated = False
    terminated = False
    while(terminated == False and truncated == False):
        action = model.predict(env)
        obs, reward, termination, truncation, info = env.step(action)
        new_obs = dh.process_data(obs)

        states.append(new_obs)
        rewards.append(reward)
        terminated = termination
        truncated = truncation
        infos.append(info)

    return states, rewards, terminated, truncated, infos

In [ ]:
data_path = 'data/train.xlsx'
simulations = 100
learning_rate = 0.8
model = TabularQLearning(data_path)
Qtable = model.train(simulations, learning_rate)

-124928.03570170737
1.0
-124584.82325111085
0.9905
-126201.483115633
0.981
-129535.03155107862
0.9715
-126592.37541890066
0.962
-127655.86963095189
0.9525
-127440.25282901313
0.9430000000000001
-129295.54503885123
0.9335
-129970.9931698305
0.924
-130317.35401273951
0.9145
-130030.64604566825
0.905
-128295.96223981437
0.8955
-129837.67662470874
0.886
-131569.17868425322
0.8765000000000001
-131650.21696089607
0.867
-131017.78524987146
0.8575
-134427.08604125658
0.848
-131200.6213605931
0.8385
-135814.20852133166
0.829
-133374.85675690015
0.8195
-132625.32944323288
0.81
-134318.75851166318
0.8005
-133237.94303477832
0.791
-133194.8066683378
0.7815
-133574.55911331382
0.772
-133741.97665770713
0.7625
-132659.24577472953
0.753
-130562.6465200645
0.7435
-133675.15038449518
0.734
-129120.7638226873
0.7245
-130105.69905547982
0.7150000000000001
-133474.82192692015
0.7055
-126959.05230918185
0.696
-129006.39390797462
0.6865
-127950.59322976645
0.677
-125942.16304629276
0.6675
-126991.3547955791